# Análise de Tópicos

In [1]:
import pandas as pd
import spacy
from bertopic import BERTopic

import re
from datetime import datetime

from gensim.utils import simple_preprocess

import nltk
nltk.download(["stopwords", "rslp"])
stopwords = nltk.corpus.stopwords.words("portuguese")

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
df_reviews = pd.read_csv("guacamole_reviews.csv")
df_reviews.head()

,Date,Review
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já..."
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest..."
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó..."


In [3]:
df_comments = df_reviews["Review"]
df_dates = df_reviews["Date"]

In [4]:
def remove_stopwords(text):
    words = simple_preprocess(text)
    phrase_adjusted = " ".join([word for word in words if word not in stopwords])
    return phrase_adjusted.lower()

spacy_lemma = spacy.load("pt_core_news_sm")

def lemmatizer(text, postags_permit=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    doc = spacy_lemma(text.lower())
    doc_lemma = " ".join([token.lemma_ for token in doc if token.pos_ in postags_permit])
    return doc_lemma

In [5]:
def parse_date(date_str):
    months = {
        'janeiro': 'January', 'fevereiro': 'February', 'março': 'March', 'abril': 'April',
        'maio': 'May', 'junho': 'June', 'julho': 'July', 'agosto': 'August',
        'setembro': 'September', 'outubro': 'October', 'novembro': 'November', 'dezembro': 'December'
    }
    match = re.match(r'(\d+) de (\w+) de (\d+)', date_str)
    if match:
        day, month, year = match.groups()
        month = months[month.lower()]
        return datetime(int(year), list(months.values()).index(month) + 1, int(day)).strftime("%Y-%m-%d")
    else:
        return None

In [6]:
df_reviews["Review Lemma"] = df_reviews["Review"].map(remove_stopwords)
df_reviews["Review Lemma"] = df_reviews["Review"].map(lemmatizer)

In [7]:
df_reviews["Date Formatted"] = df_reviews["Date"].apply(parse_date)

In [8]:
df_reviews.head()

,Date,Review,Review Lemma,Date Formatted
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27


In [9]:
df_comments = df_reviews["Review Lemma"]
doc_comments = df_comments.to_list()
doc_comments[:5]

['atendimento maravilhoso matheu muito atencioso colocar playlist atendente super simpatico',
 'gostar muito prato atendimento solicito comida saboroso certeza voltar mais vez',
 'ameir bom cozinha mexicano já comi espaço muito animar imersivo decoração casa linda atendimento excelente sempre',
 'bom ambiente decoração funcionário prestativo comida boa música agradável divertido mariachi',
 'amoma atendimento matheu bom amar atendimento sensacional comida muito saboroso']

In [10]:
topic_model = BERTopic(language="portuguese", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(doc_comments)

2023-12-10 18:29:06,429 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/90 [00:00<?, ?it/s]

2023-12-10 18:30:18,935 - BERTopic - Embedding - Completed ✓
2023-12-10 18:30:18,937 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-10 18:30:56,098 - BERTopic - Dimensionality - Completed ✓
2023-12-10 18:30:56,111 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-10 18:30:56,565 - BERTopic - Cluster - Completed ✓
2023-12-10 18:30:56,580 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-10 18:30:56,757 - BERTopic - Representation - Completed ✓


In [11]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,225,-1_muito_bem_bom_atendimento,"[muito, bem, bom, atendimento, comida, mais, l...",[ambiente bem animar comida muito bom além ter...
1,0,733,0_mexicano_guacamole_comida_restaurante,"[mexicano, guacamole, comida, restaurante, mai...",[comer excelente ambiente bem característico m...
2,1,657,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[ambiente alegre descontrair bom comida atendi...
3,2,347,2_música_bom_muito_ambiente,"[música, bom, muito, ambiente, atendimento, mu...",[ambiente muito agradável música bom excelente...
4,3,238,3_aniversário_comemorar_amigo_muito,"[aniversário, comemorar, amigo, muito, bom, lu...",[bom atendimento bom querer comemorar aniversá...


In [12]:
topic_model.get_topic(0)

[('mexicano', 0.07472017827493527),
 ('guacamole', 0.03960115750776117),
 ('comida', 0.03627796523088445),
 ('restaurante', 0.03570624909413835),
 ('mais', 0.03418303233185723),
 ('bom', 0.033548886619954504),
 ('muito', 0.030257563346517625),
 ('ter', 0.026696346377773558),
 ('bem', 0.02568179278230327),
 ('ambiente', 0.025045067358162077)]

In [15]:
df_topics = df_reviews.copy()

df_topics["Topics"] = topics

topic_name = freq.drop(columns=["Count"]).rename(columns={"Topic": "Topics", "Name": "Names"})
df_topics = df_topics.merge(topic_name, how="left")

df_topics.head()

,Date,Review,Review Lemma,Date Formatted,Topics,Names,Representation,Representative_Docs
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[ambiente alegre descontrair bom comida atendi...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[ambiente alegre descontrair bom comida atendi...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01,0,0_mexicano_guacamole_comida_restaurante,"[mexicano, guacamole, comida, restaurante, mai...",[comer excelente ambiente bem característico m...
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29,2,2_música_bom_muito_ambiente,"[música, bom, muito, ambiente, atendimento, mu...",[ambiente muito agradável música bom excelente...
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[ambiente alegre descontrair bom comida atendi...


In [ ]:
topic_model.visualize_barchart(top_n_topics=20)

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
dates_reviews = df_reviews["Date Formatted"].to_list()

In [ ]:
topics_over_time = topic_model.topics_over_time(doc_comments, dates_reviews, datetime_format="%Y-%m-%d", nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

# Análise de Sentimentos

In [16]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForSequenceClassification  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline

model = AutoModelForSequenceClassification.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student')
tokenizer = AutoTokenizer.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student', do_lower_case=False)
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [17]:
sentiment_task("eu sou feliz")

[{'label': 'positive', 'score': 0.9540010094642639}]

In [18]:
%%time

df_topics["Sentiment"] = df_topics["Review"].apply(lambda x: sentiment_task(x)[0]["label"])
df_topics["Sentiment Score"] = df_topics["Review"].apply(lambda x: sentiment_task(x)[0]["score"])    

CPU times: total: 46min 46s
Wall time: 7min 55s


In [19]:
df_topics.head()

,Date,Review,Review Lemma,Date Formatted,Topics,Names,Representation,Representative_Docs,Sentiment,Sentiment Score
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[ambiente alegre descontrair bom comida atendi...,positive,0.989543
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[ambiente alegre descontrair bom comida atendi...,positive,0.626670
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01,0,0_mexicano_guacamole_comida_restaurante,"[mexicano, guacamole, comida, restaurante, mai...",[comer excelente ambiente bem característico m...,positive,0.967855
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29,2,2_música_bom_muito_ambiente,"[música, bom, muito, ambiente, atendimento, mu...",[ambiente muito agradável música bom excelente...,positive,0.940317
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27,1,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[ambiente alegre descontrair bom comida atendi...,positive,0.979101


In [20]:
df_topics.to_csv('guacamole_topics_sentiment.csv', index=False)

In [23]:
count_sentiment = df_topics["Sentiment"].value_counts()
count_sentiment

Sentiment
positive    2648
negative     218
neutral        6
Name: count, dtype: int64